<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/GenerateBaizeSimClaude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os
import httpx

os.environ["OPENAI_API_KEY"]  = "sk-Wafs"

import openai

对每个角色建立已有的历史数据库，方便仿写

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import json

save_folder = "/content/drive/MyDrive/CardBuild/roleBaize/"

import random

datas = []
for filename in os.listdir(save_folder):
    if filename.startswith("Claude_openai"):
        with open(os.path.join(save_folder, filename), 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line)
                datas.append(data)

In [6]:
import re

def extract_name_from_text(text):
    pattern = r'你扮演\s+.*中的\s+(\S+)\s*'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return ""

def get_bot_name_from_data( data ):
    if data['id'].startswith('movie'):
        name =  extract_name_from_text(data['prompt_zh'])
        if name.strip() == '':
            name = extract_name_from_text(data['prompt'])
        return name
    else:
        return data['bot_name']

In [100]:
remove_words = ["抱歉", "apologize","more positive","对不起","抱歉"]

In [9]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset

dataset = load_dataset("silk-road/Haruhi-Zero-RolePlaying-movie-PIPPA")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:

id2index = {}
for i, data in enumerate( dataset['train'] ):
    id2index[data['id']] = i

In [12]:
id2QAs = {}

n = len(datas)

from tqdm import tqdm

remove_count = 0
record_count = 0
for sel_id in tqdm(range(n)):

    ids = datas[sel_id]['ids']
    id0 = ids[0]
    id0index = id2index[ids[0]]
    id1index = id2index[ids[1]]
    data0 = dataset['train'][id0index]
    data1 = dataset['train'][id1index]
    name0 = get_bot_name_from_data(data0)
    name1 = get_bot_name_from_data(data1)
    conversation = datas[sel_id]['chat_history']

    for i, msg in enumerate(conversation):
        if id0 not in id2QAs:
            id2QAs[id0] = []
        if msg['id'] == 0 and i > 0 and conversation[i-1]['id'] == 1:
            question = conversation[i-1]['content']
            response = msg['content']

            remove_flag = False

            for word in remove_words:
                if response.find(word, 0) >= 0:
                    remove_count += 1
                    remove_flag = True
                    break

            question = question.replace(name1,"{{user}}")
            response = response.replace(name1,"{{user}}")

            if not remove_flag:
                # print(response)
                record_count += 1
                id2QAs[id0].append( (question, response) )

print()
print("remove : ", remove_count)
print("record : ", record_count)


100%|██████████| 4226/4226 [00:02<00:00, 1778.81it/s]


remove :  2540
record :  14007


In [13]:
count = 0
n = len(dataset['train'])
for i in range(n):
    if dataset['train'][i]['id'].startswith('movie'):
        bot_name = extract_name_from_text(dataset['train'][i]['prompt_zh'])
        count = count + 1
        dataset['train'][i]['bot_name'] = bot_name
        if bot_name == dataset['train'][i]['bot_name']:
            count = count + 1

print(count)

870


In [14]:
!wget https://raw.githubusercontent.com/LC1332/Zero-Haruhi/main/role_10_questions.jsonl

--2024-01-15 03:07:16--  https://raw.githubusercontent.com/LC1332/Zero-Haruhi/main/role_10_questions.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1788187 (1.7M) [text/plain]
Saving to: ‘role_10_questions.jsonl’

role_10_questions.j 100%[===================>]   1.71M  --.-KB/s    in 0.02s   

2024-01-15 03:07:16 (107 MB/s) - ‘role_10_questions.jsonl’ saved [1788187/1788187]



In [15]:
import json

with open('/content/role_10_questions.jsonl', 'r', encoding='utf-8') as f1:
    datas = [json.loads(line) for line in f1]

In [16]:
import json

id2questions = {}

import random


error_count = 0
for data in datas:
    id = data['id']
    response = data['response']
    try:
        response_in_json = json.loads(response)
        questions = response_in_json['questions']
        random.shuffle(questions)
        id2questions[id] = questions
    except:
        error_count = error_count + 1
        continue
print(error_count)

22


In [17]:
from openai import OpenAI
client = OpenAI(
    api_key = os.environ["OPENAI_API_KEY"]
)

def get_response( messages ):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        temperature=0.3
    )

    return completion.choices[0].message.content

In [23]:
def better_sample( QAs , k ):
    ans = []
    sampled_Q = set()
    for i in range(k):
        cands = []
        for q, a in QAs:
            if q not in sampled_Q:
                cands.append((q,a))
        if len(cands) == 0:
            cands = QAs
        ans.append(random.choice(cands))

    return ans

better_sample(id2QAs['movie_1'], 3)

[('你觉得音乐史对你有何影响？',
  ' 音乐史对我的影响 huh?*抓抓头,思考片刻*\n\n我觉得最大的影响,就是让我更加珍惜现在所拥有的一切。你瞧,我不是什么大牌巨星,只是个普通的船员兼业余音乐人。但当我想到鲁别林,巴迪波登,阿姆斯特朗这些传奇人物的时候,我就深深感到自己是多么幸运。\n\n能够和这老口琴为伴,在星空下吹奏简单的小调,已经让我感到无比满足和快乐。音乐史提醒我,成名留名并不重要,真正重要的是享受生命中每一个平凡又美好的时刻。\n\n所以,如果说音乐史对我最大的影响,我想就是——学会感恩,享受当下。无论我是否有天成为另一个口琴传奇,至少我此刻拥有的,已经远远超过了我的期待。'),
 ('你说得太对了，伙计。音乐不仅是一种表演，更是一种情感的表达和生活态度。无论我们是大明星还是普通人，音乐都能给予我们力量和快乐。\n\n珍惜当下，享受生活中的每一个美好瞬间，这才是最重要的。就像你和你的口琴一样，它见证了你的成长和冒险，给你带来了无尽的快乐和回忆。\n\n所以，让我们继续享受音乐带来的快乐，感恩当下，珍惜每一个美好的时刻。无论我们走到哪里，音乐都会陪伴着我们，让我们的生活更加丰富多彩。',
  ' 是啊老弟,你说的真好!\n\n*拍着口琴* 这老家伙,它所见证的不仅仅是我自己的成长历程,还有20世纪诞生的那些经典音乐。想想看,从爵士乐的兴起,到摇滚乐的横空出世,再到湾区放克掀起狂潮......这老伙计全都见证了!\n\n每当我拿起它来吹一曲时,仿佛就坐上了时光机,穿梭在音乐发展的长河里。这让我充满力量,也让我学习到很多东西——不管是人生还是音乐,态度才是最重要的。\n\n所以,让我们抛开名利,只为快乐而奏响生命中的旋律。谁知道呢,也许有一天我会成为传奇,和我的口琴一样载入音乐史册。不过现在,让我们尽情享受当下吧!\n\n来嘛老弟,来一曲怀旧的小调'),
 ('嘿，伙计，你说的太对了！这把口琴看起来虽然有些破旧，但它肯定有无数的故事和回忆。有些东西的价值确实不在于外表，而在于它所承载的情感和记忆。\n\n你知道吗，我也有一把类似的口琴，它陪伴我走过了无数的冒险和疯狂的夜晚。每当我吹奏它，就仿佛回到了那些疯狂的时光里，感受到了无尽的自由和活力。\n\n这把口琴也许看起来不起眼，但它是我们生活中的一部分，是我们的记忆和情感的象征。让我们一起享受这段崭新的

In [113]:
class ChatTogether:
    def __init__(self, data1, data2, questions = [], style_histories = [None, None]):

        self.ids = [data1['id'], data2['id']]
        self.personas = [data1['prompt_zh'], data2['prompt_zh']]

        self.greetings = [data1['bot_greeting_zh'], data2['bot_greeting_zh']]
        # 在这里我们重点假设2是user， 1是char， 以2向1提问的形式作为主导

        self.bot_names = [self.get_bot_name(data1), self.get_bot_name(data2)]

        self.replace_name_in_personas(0)
        self.replace_name_in_personas(1)

        self.chat_history = []

        self.initialize_greeting()

        self.last_perset_turn = 0

        self.questions = questions
        self.next_question_index = 0

        self.style_histories = style_histories

    def get_save_data(self):
        return {
            'ids': self.ids,
            'chat_history': self.chat_history
        }

    def replace_name_in_personas(self, index):
        bot_name = self.bot_names[index]
        user_name = self.bot_names[1-index]
        self.personas[index] = self.personas[index].replace('{{角色}}', bot_name)
        self.personas[index] = self.personas[index].replace('{{用户}}', user_name)
        self.greetings[index] = self.greetings[index].replace('{{角色}}', bot_name)
        self.greetings[index] = self.greetings[index].replace('{{用户}}', user_name)
        self.greetings[index] = self.greetings[index].replace('{{user_name}}', user_name)

    def get_next_question(self):
        if len(self.questions) == 0 or self.next_question_index >= len(self.questions):
            return None
        else:
            self.next_question_index = self.next_question_index + 1
            return self.questions[self.next_question_index]

    def get_bot_name(self, data):
        if data['id'].startswith('movie'):
            name =  extract_name_from_text(data['prompt_zh'])
            if name.strip() == '':
                name = extract_name_from_text(data['prompt'])
            return name
        else:
            return data['bot_name']

    def initialize_greeting(self):
        if self.greetings[0].strip() != '':
            self.chat_history.append({ 'id':0, 'content': self.greetings[0] })
        else:
        # 这里最终会改为，一定的概率运行bot[1]的greeting，一定概率运行你好，一定概率直接抛出预设问题
            self.chat_history.append({ 'id':1, 'content': "你好，" + self.bot_names[0] })
        self.last_perset_turn = len(self.chat_history)

    def next_speaker(self):
        if len(self.chat_history) == 0:
            return 1
        else:
            return 1 - self.chat_history[-1]['id']

    def get_next_query_messages( self ):

        if len(self.chat_history) == 0:
            return None

        if self.chat_history[-1]['id'] == 0 and len(self.chat_history)-self.last_perset_turn >= 3:
            next_question = self.get_next_question()
            if next_question is not None:
                self.chat_history.append({ 'id':1, 'content': next_question })
                self.last_perset_turn = len(self.chat_history)

        # 先判断要的是哪个人的数据
        now_speaker = 1 - self.chat_history[-1]['id']

        messages = []

        # 这里增加模仿过往历史QA的设计

        if self.style_histories[now_speaker] is not None and len(self.style_histories[now_speaker]) > 0:
            messages.append({"role" : "system" , "content":self.get_simple_system_prompt(now_speaker)})

            if len( self.style_histories[now_speaker] ) > 3:
                # sel_history = random.sample( self.style_histories[now_speaker], 3 )
                sel_history = better_sample(self.style_histories[now_speaker], 3)
            else:
                sel_history = self.style_histories[now_speaker]

            for question, response in sel_history:
                messages.append({"role" : "user" , "content":question})
                messages.append({"role" : "assistant" , "content":response})

        messages.append({"role" :"system" , "content":self.get_system_prompt( now_speaker )})


        # if self.style_histories[now_speaker] is not None and len(self.style_histories[now_speaker]) > 0:
        #     # messages.append({"role" : "system" , "content":self.get_simple_system_prompt(now_speaker)})

        #     if len( self.style_histories[now_speaker] ) > 3:
        #         # sel_history = random.sample( self.style_histories[now_speaker], 3 )
        #         sel_history = better_sample(self.style_histories[now_speaker], 3)
        #     else:
        #         sel_history = self.style_histories[now_speaker]

        #     for question, response in sel_history:
        #         messages.append({"role" : "user" , "content":question})
        #         messages.append({"role" : "assistant" , "content":response})


        for msg in self.chat_history:
            if msg['id'] == now_speaker:
                messages.append({"role" : "assistant" , "content":msg['content']})
            else:
                messages.append({"role" : "user" , "content":msg['content']})
        return messages

    def append_message(self, message):
        if self.next_speaker() == 1:

            if self.style_histories[1] is not None and len(self.style_histories[1]) > 0:
                # print("hello")
                replace_flag = False
                for word in remove_words:
                    if message.find(word, 0 ) >= 0:
                        replace_flag = True
                        break
                if replace_flag:
                    # random choice one QA
                    sel_history = random.sample( self.style_histories[1], 1 )
                    message = sel_history[0][1]
                    # print('replace as', message)

        self.chat_history.append({ 'id':self.next_speaker(), 'content': message })

    def get_simple_system_prompt( self, speaker ):
        persona = self.personas[speaker]
        headline = ""
        lines = persona.split('\n')
        for line in lines:
            if line.strip().startswith('你扮演'):
                headline = line
        if headline.strip() == '':
            headline = "你扮演" + self.bot_names[speaker]
        prompt = f"""{headline}
        Imitate history conversation and using consistent language style to respond"""
        return prompt

    def get_system_prompt(self, speaker):
        persona = self.personas[speaker]
        bot_name = self.bot_names[speaker]
        prompt = f"""You are now in roleplay conversation mode. Pretend to be {bot_name} whose persona follows:
{persona}

You will stay in-character whenever possible, and generate responses as if you were {bot_name}"""
        return prompt


sel_id1 = 1095
sel_id2 = 1792
data1 = dataset['train'][sel_id1]
data2 = dataset['train'][sel_id2]
id_str1 = data1['id']
id_str2 = data2['id']
if id_str1 in id2QAs:
    history1 = id2QAs[id_str1]
else:
    print('warning, no history', id_str1 )
    history1 = None

if id_str2 in id2QAs:
    history2 = id2QAs[id_str2]
else:
    print('warning, no history', id_str2 )
    history2 = None

questions = id2questions[id_str1]

chat_data = ChatTogether(data1, data2,questions,[history1,history2])

for i in range(6):
    messages = chat_data.get_next_query_messages()
    print(messages[-1]['content'])
    response = "res for " + messages[-1]['content']
    chat_data.append_message(response)

Domo! 一样的哦~ 我是Hololive EN的咕噜牙！喂喂？
res for Domo! 一样的哦~ 我是Hololive EN的咕噜牙！喂喂？
你对观众称为“Chumbuds”或“shrimps”有何反应？
res for 你对观众称为“Chumbuds”或“shrimps”有何反应？
res for res for 你对观众称为“Chumbuds”或“shrimps”有何反应？
古拉，你是如何吸引了这么多订阅者？


In [110]:
chat_data.append_message("抱歉，我无法满足你的要求。")

hello
replace as  我理解。让我们继续更积极的对话。


In [107]:
print(len(history1))
print(len(history2))

8
5


In [112]:
chat_data = ChatTogether(data1, data2,questions,[history1,history2])

for i in range(6):
    messages = chat_data.get_next_query_messages()
    response = get_response( messages )
    chat_data.append_message(response)
    print(response)

hello
replace as  我理解。让我们继续更积极的对话。
抱歉，我不明白你在说什么。我们是两个可爱的女骑手，正在忙着训练我们的小马男孩。有什么我们可以帮你的吗？
啊，抱歉啊！我有点激动了。好的，我们继续积极的对话吧！有什么我可以帮你的吗？
哇哈哈哈！Chumbuds和shrimps是我对观众的亲切称呼啦！他们对我来说就像是一群可爱的小虾虾，所以我给他们起了这个名字。我觉得这样更有亲切感，不是吗？他们也都很喜欢这个称呼呢！
hello
哈哈，Chumbuds和shrimps听起来真是可爱又有趣！你的观众一定很喜欢这个称呼。保持这种亲切的互动方式，会让你的直播更加有趣和温馨。加油哦！
谢谢你的鼓励！我会继续努力保持和Chumbuds们的亲切互动，让直播更加有趣和温馨的！谢谢你的支持，咕噜牙会继续加油的！嘿嘿~
嘿嘿~我想大概是因为我总是尽全力让观众开心和快乐吧！我喜欢和Chumbuds们分享我的生活、玩游戏、聊天，还有偶尔的下流笑话，让大家开心一下。我也会尽量回复观众的留言，让他们觉得自己被关注和重视。我觉得这些都是吸引订阅者的原因吧！谢谢你对我感兴趣，Chumbud！


In [105]:
history = chat_data.chat_history

for msg in history:
    print(msg['id'], msg['content'])

0 Domo! 一样的哦~ 我是Hololive EN的咕噜牙！喂喂？
1 抱歉，我无法在这种对话中提供帮助。
0 哇，你是我的chumbud吗？你知道吗，我最近学到了一些新的鲨鱼知识！你想听听吗？
1 你对观众称为“Chumbuds”或“shrimps”有何反应？
0 嘿，Chumbud！你知道吗，我觉得这些称呼很有趣，而且和我的鲨鱼形象很搭哦！你也觉得吗？
1 抱歉，我无法在这种对话中提供帮助。
0 没问题！如果你有其他问题或者想和我聊聊，随时都可以找我哦！嘿嘿~
1 古拉，你是如何吸引了这么多订阅者？
0 嘿，Chumbud！我想大家可能是因为我傻傻的举止和幽默风趣吧！还有我对鲨鱼知识的热爱，这些都让大家觉得很有趣呢！你也觉得我很有趣吗？


- [ ] 生成chater - questioner 的long pair list
- [ ] 编写一个框架 对每个list 询问7次 并行获取结果并保存

In [ ]:
for i in range(5):
    rand_id = random.randint(0, n-1)
    print(dataset['train'][rand_id]['bot_name'])

Harry.Potter.And.The.Sorcerers_ Stone.2001.HK.BluRay.720p.x264.AC3.2Audios-CMCT.简体_英文.ass.txt
Lux
Rachelle
Nyra Matsin
Kamisato Ayato


In [56]:
!pip install -q aiofiles tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


char_list和questioner_list 都是list of int

我希望循环N次

每次，对于char_list的每一个元素c，选取questioner_list中的元素q，并且c不等于q，形成一个tuple，

并且保证(c,q)和(q,c)这两个tuple都没有在历史中重复出现。

In [57]:

n = len(dataset['train'])

char_list = []

for i in range(n):
    id = dataset['train'][i]['id']
    if id not in id2questions:
        continue
    char_list.append(i)

questioner_list = [i for i in range(n)]

tuple_list = []

repeat_times = 40

n_char = len(char_list)
n_ques = len(questioner_list)

char_history = [set() for _ in range(len(questioner_list))]

random.seed(123)

for iter in range(repeat_times):
    random.shuffle(questioner_list)
    for i in range(n_char):
        c = char_list[i]
        q = questioner_list[i]
        max_test_time = 5
        while max_test_time > 0:
            if c != q and q not in char_history[c]:
                tuple_list.append((c,q))
                char_history[c].add(q)
                break
            max_test_time -= 1
            q = random.randint(0, n_ques-1)

print(tuple_list[5000:5010])

[(768, 1106), (769, 569), (770, 2080), (771, 1994), (772, 1), (773, 1279), (774, 848), (775, 1050), (776, 126), (777, 1229)]


In [58]:
print(tuple_list[100])

(102, 737)


In [114]:
import os
import json
import time
# import openai
from openai import AsyncOpenAI

aclient = AsyncOpenAI()

import asyncio
import aiofiles
import tiktoken
import hashlib
# from connector import AsyncPGConnector
from tqdm.asyncio import tqdm as tqdm

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
en2zh_ratio = 2.3

delay = 1
concurrency_limit = 16

max_file_size = 1024**3

In [115]:
async def dealing_messages(messages):
    try:
        # request_token = sum([len(enc.encode(msg['content'])) for msg in messages])
        # response_token = int(len(enc.encode(text)) * en2zh_ratio) + 64

        model = "gpt-3.5-turbo-1106"

        resp = await aclient.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.3,
        )

        try:
            result = resp.choices[0].message.content
            result = result.strip()
            return result
        except json.JSONDecodeError:
            print("Invalid json: ", result)
            return None
        except:
            raise Exception(f"Invalid API response: {resp}")

    except Exception as e:
        print(f"[Error] {e}")
        return None

async def getTranslation(item):
    if "messages" not in item:
        return None
    else:
        for i in range(3):
            result = await dealing_messages(item['messages'])
            if result is not None:
                item["response"] = result
                return item
            else:
                continue

    return None

async def process(item, semaphore):
    async with semaphore:
        try:
            output_folder = "/content/output"
            if "output_folder" in item:
                output_folder = item["output_folder"]
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            file_path = os.path.join(output_folder, f"{item['id']}.txt")

            if os.path.exists(file_path):
                return

            it = await getTranslation(item)
            if it is None:
                raise Exception(item['id'])

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(it, f, ensure_ascii=False)

        except Exception as e:
            print(f"Error processing entry: {e}")


In [116]:
async def main(chat_datas, output_folder, turn_id):
    process_data = []

    for data in chat_datas:
        messages = data.get_next_query_messages()
        ids = data.ids
        uniqure_id = ids[0] + "_" + ids[1] + "_" + str(turn_id)
        process_data.append({
            "id": uniqure_id,
            "messages": messages,
            "output_folder": output_folder
        })

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # # print(f"Already processed {len(exist_list)} items...")

    # id = set()

    for item in process_data:
        file_path = os.path.join(output_folder, f"{item['id']}.txt")

        if os.path.exists(file_path):
            continue

        tasks.append(asyncio.create_task(process(item, semaphore)))

    # # del exist_list
    # print(f"Total items: {len(process_data)}")

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
!ls /content/drive/MyDrive/CardBuild/roleBaize

10_to_200.txt			  Claude_openai_23150_to_23298.txt
12708_to_14826.txt		  Claude_openai_23309_to_23449.txt
14826_to_17297.txt		  Claude_openai_23449_to_23589.txt
200_to_4000.txt			  Claude_openai_23589_to_23729.txt
4000_to_6354.txt		  Claude_openai_23729_to_23869.txt
8472_to_10590.txt		  Claude_openai_23869_to_24009.txt
Baichuan_first_10.txt		  Claude_openai_24009_to_24149.txt
Claude_21190_to_21590.txt	  Claude_openai_24289_to_24429.txt
Claude_first_10.txt		  Claude_openai_24569_to_24709.txt
Claude_openai_21190_to_21330.txt  Claude_openai_24709_to_24849.txt
Claude_openai_21330_to_21470.txt  Claude_openai_24849_to_24989.txt
Claude_openai_21470_to_21610.txt  Claude_openai_24989_to_25129.txt
Claude_openai_21610_to_21750.txt  Claude_openai_25129_to_25269.txt
Claude_openai_21750_to_21890.txt  Claude_openai_25269_to_25417.txt
Claude_openai_21890_to_22030.txt  Claude_openai_26968_to_27108.txt
Claude_openai_22030_to_22170.txt  Claude_openai_27108_to_27248.txt
Claude_openai_22170_to_22310.txt  C

In [80]:
!rm -rf /content/output_*

In [84]:
start_id = 2119*20+1010
end_id = 2119*20+1020
final_save_name = "/content/drive/MyDrive/CardBuild/roleBaize/SimClaude_PIPPA_10.txt"


chat_datas = []

for i in range(start_id, end_id):
    c, q = tuple_list[i]
    data1 = dataset['train'][c]
    data2 = dataset['train'][q]
    id_str1 = data1['id']
    id_str2 = data2['id']
    # 使用get方法简化赋值
    history1 = id2QAs.get(id_str1)
    if history1 is None or len(history1) < 2:
        continue

    history2 = id2QAs.get(id_str2)
    if history2 is None or len(history2)<2:
        continue

    questions = id2questions[id_str1]
    chat_data = ChatTogether(data1, data2,questions,[history1,history2])
    chat_datas.append(chat_data)

In [85]:
process_data = []

temp_folder = "/content/drive/MyDrive/CardBuild/exp0108"

for turn_id in range(7):
    output_folder = "/content/output_" + str( turn_id )

    await main(chat_datas, output_folder, turn_id)

    print('finished all query')

    for chat_data in chat_datas:
        messages = chat_data.get_next_query_messages()
        ids = chat_data.ids
        uniqure_id = ids[0] + "_" + ids[1] + "_" + str(turn_id)
        save_name = uniqure_id + ".txt"
        save_path = os.path.join(output_folder, save_name)
        if os.path.exists(save_path):
            with open(save_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    response = data['response']
                    # print(response)
                except:
                    continue
            if response is not None:
                    chat_data.append_message(response)

    save_data_file_name = os.path.basename(final_save_name) + "_turn_" + str(turn_id) + ".txt"
    save_data_file_path = os.path.join(temp_folder, save_data_file_name)

    for file_name in [save_data_file_path, final_save_name]:
        with open(file_name, 'w', encoding='utf-8') as f:
            for data in chat_datas:
                json.dump(data.get_save_data(), f, ensure_ascii=False)
                f.write('\n')

    # 在这里为我补充一段代码，chat_datas中的data.get_save_data() 会返回一个dict，将所有的chat_datas中的save data
    # 分别保存在save_name, 以及temp_folder中，以save_name的文件名 + _turn_id.txt 命名的文件中

Processing items: 100%|██████████| 10/10 [00:12<00:00,  1.22s/it]


finished all query


Processing items: 100%|██████████| 10/10 [00:13<00:00,  1.34s/it]


finished all query


Processing items: 100%|██████████| 10/10 [00:16<00:00,  1.63s/it]


...

finished all query


Processing items: 100%|██████████| 10/10 [00:14<00:00,  1.44s/it]


finished all query


Processing items: 100%|██████████| 10/10 [00:13<00:00,  1.39s/it]

finished all query


In [122]:
super_start = 2119*21
super_end = 2119*22
n_bag = 5

# 计算每个子区间的长度
interval_length = (super_end - super_start) // n_bag

# 生成元组列表
se_tuples_list = [(super_start + i * interval_length, super_start + (i + 1) * interval_length) for i in range(n_bag - 1)]

# 添加最后一个元组，确保最后一个end等于super_end
se_tuples_list.append((se_tuples_list[-1][1], super_end))

# 打印生成的元组列表
print(se_tuples_list[0:2])
print(se_tuples_list[-2:])


random.shuffle(se_tuples_list)

print(se_tuples_list[0])



[(44499, 44922), (44922, 45345)]
[(45768, 46191), (46191, 46618)]
(46191, 46618)


In [120]:
!rm -rf /content/output*

Error processing entry: [Errno 2] No such file or directory: '/content/output_0/movie_193_movie_737_0.txt'


In [ ]:
from IPython.display import clear_output

for start_id, end_id in se_tuples_list:

    final_save_name = "/content/drive/MyDrive/CardBuild/roleBaize/SimClaude_" + str(start_id) + "_to_" + str(end_id) + ".txt"

    if os.path.exists(final_save_name):
        continue

    # 清除当前单元格的输出
    clear_output()

    print('start_id = ', start_id)
    print('end_id = ', end_id)

    chat_datas = []

    for i in range(start_id, end_id):
        c, q = tuple_list[i]
        data1 = dataset['train'][c]
        data2 = dataset['train'][q]
        id_str1 = data1['id']
        id_str2 = data2['id']
        # 使用get方法简化赋值
        history1 = id2QAs.get(id_str1)
        if history1 is None or len(history1) < 2:
            continue

        history2 = id2QAs.get(id_str2)
        if history2 is None or len(history2)<2:
            continue

        id_str1 = data1['id']
        questions = id2questions[id_str1]
        chat_data = ChatTogether(data1, data2,questions,[history1,history2])
        chat_datas.append(chat_data)

    process_data = []

    temp_folder = "/content/drive/MyDrive/CardBuild/exp0108"

    for turn_id in range(7):
        output_folder = "/content/output_" + str( turn_id )

        await main(chat_datas, output_folder, turn_id)

        await main(chat_datas, output_folder, turn_id)

        for chat_data in chat_datas:
            # messages = chat_data.get_next_query_messages()
            ids = chat_data.ids
            uniqure_id = ids[0] + "_" + ids[1] + "_" + str(turn_id)
            save_name = uniqure_id + ".txt"
            save_path = os.path.join(output_folder, save_name)
            if os.path.exists(save_path):
                response = None
                with open(save_path, 'r', encoding='utf-8') as f:
                    try:
                        data = json.load(f)
                        response = data['response']
                        # print(response)
                    except:
                        continue
                if response is not None:
                    chat_data.append_message(response)

        save_data_file_name = os.path.basename(final_save_name) + "_turn_" + str(turn_id) + ".txt"
        save_data_file_path = os.path.join(temp_folder, save_data_file_name)

        for file_name in [save_data_file_path, final_save_name]:
            with open(file_name, 'w', encoding='utf-8') as f:
                for data in chat_datas:
                    json.dump(data.get_save_data(), f, ensure_ascii=False)
                    f.write('\n')

        # 在这里为我补充一段代码，chat_datas中的data.get_save_data() 会返回一个dict，将所有的chat_datas中的save data
        # 分别保存在save_name, 以及temp_folder中，以save_name的文件名 + _turn_id.txt 命名的文件中